# Tutorial 01: Introduction to Parla

This tutorial introduces what Parla is and some of its core abstractions such as 'Tasks', 'TaskSpaces', 'Devices'. 


## Installation

Hopefully you have already installed Parla as described in the [README](../../../README.md) or are using a provided Docker container.
If not, please do so now.

To install Parla, you will need to clone the repository and install it using pip.
```bash
git clone https://github.com/ut-parla/parla-experimental.git
cd parla-experimental
git submodule update --init --recursive
pip install -e .
```


# What is Parla?

Parla is a Python library for parallel programming. It is designed to make it easy to write parallel programs that can run on a variety of parallel hardware, including multi-core CPUs, GPUs, on a single node. 

It is a single-process thread-based runtime for Task-Based parallel programming. 

In [23]:
from parla import Parla
from parla.tasks import spawn, TaskSpace 
from parla.devices import cpu, gpu 

from typing import Callable
from time import perf_counter

In [24]:
def run(function: Callable[[], float], print_time: bool =False):
    # Start the Parla runtime
    with Parla():
        
        #Create an encapsulating top-level task to kick off the computation and wait for it to complete.
        @spawn(placement=cpu, vcus=0)
        async def top_level_task():
            
            # Run the Parla application and print the time it took if requested.
            start_t = perf_counter()
            await function()
            end_t = perf_counter()
            
            elapsed = end_t - start_t
            if print_time:
                print(f"Execution time: {elapsed} seconds", flush=True)
            return elapsed

In [40]:
async def first_example():
    
    @spawn()
    def task_hello():
        print("Hello World!", flush=True)
        
    @spawn()
    def task_goodbye():
        print("Goodbye World!", flush=True)


run(first_example)

Hello World!
Goodbye World!


In [ ]:
async def serial_example():
    data = np.zeros(1)

    T = TaskSpace("My First TaskSpace")
    for i in range(5):

        @spawn(T[i], dependencies=[T[i - 1]])
        def task():
            print("Hello from task", i, "data =", data[0], flush=True)
            data[0] += 1


run(serial_example)

In [57]:
from numba import njit
import numpy as np

import pykokkos as pk 
pk.set_default_space(pk.OpenMP)

@pk.workunit
def daxpy_kernel(
    tid: int, 
    start: int,
    end: int,
    out: pk.View1D[float],
    a: float,
    x: pk.View1D[float],
    y: pk.View1D[float],
    stride: int = 1
):
    for i in range(start+tid, end, stride):
        out[i] = a * x[i] + y[i]
        
def daxpy(start: int, end: int, out, a: float, x, y):
    num_threads = 1
    pk.parallel_for(num_threads, daxpy_kernel, start=start, end=end, out=out, a=a, x=x, y=y, stride=num_threads)

def compile_daxpy():
    N = 100
    x = np.random.rand(N)
    y = np.random.rand(N)
    out = np.empty_like(x)
    daxpy(0, N, out, 2.0, x, y)


compile_daxpy()


async def daxpy_example():
    N = 200000000
    x = np.random.rand(N)
    y = np.random.rand(N)
    out = np.empty_like(x)
    truth = np.empty_like(x)
    a = 2.0
    splits = 2

    start_t = perf_counter()
    # truth[:] = a * x[:] + y[:]
    daxpy(0, N, truth, a, x, y)
    end_t = perf_counter()
    print("Reference: ", end_t - start_t)

    start_t = perf_counter()
    T = TaskSpace("Daxpy")
    for i in range(splits):

        @spawn(T[i], placement=cpu, vcus=0)
        def daxpy_task():
            start = i * N // splits
            end = (i + 1) * N // splits
            # out[start:end] = a * x[start:end] + y[start:end]
            daxpy(start, end, out, a, x, y)

    @spawn(T[splits], dependencies=[T[:splits]], placement=cpu, vcus=0)
    def check():
        end_t = perf_counter()
        print("Parla: ", end_t - start_t)
        print("Check: ", np.allclose(out, truth))

    await T


run(daxpy_example)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ipykernel_93/39138626.py'

In [ ]:
async def spawn_example():
    T = TaskSpace("My First TaskSpace")

    @spawn(T[1], dependencies=[T[0]])
    def task_world():
        print("World!")

    @spawn(T[0])
    def task_hello():
        print("Hello", end=" ")


run(spawn_example)